In [1]:
import glob
import os
import pickle
import sys
from tqdm import tqdm
from pathlib import Path 

# change these to your local paths
sys.path.append('~/dev/pathml') 
sys.path.append('~/dev/pathml/Pytorch-UNet')

from pathml.slide import Slide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import torch
import torchvision
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve
import time

print("Imports complete")


Imports complete


In [2]:
def caseRandomizer():
    wsi_path = Path("/pml/wsi_data/")
    dirs = os.listdir(wsi_path)
    print(f"Loaded {len(dirs)} images from {wsi_path} \r\n")
    res = np.char.rstrip(dirs, '.tif')
    df = pd.DataFrame(res)
    train_cases, val_cases, test_cases = np.split(df.sample(frac=1, random_state=123), [int(.6*len(df)), int(.8*len(df))])

    train_cases = train_cases[0].tolist()
    val_cases = val_cases[0].tolist()
    test_cases = test_cases[0].tolist()

    print("Training dataset: ", sorted(train_cases), "\r\n")
    print("Validation dataset: ", sorted(val_cases), "\r\n")
    print("Ttest: ", sorted(test_cases), "\r\n")

    return train_cases, val_cases, test_cases


In [3]:
# Since images and annotations are inputs, we load them from out of the working directory
wsi_path = Path("/pml/wsi_data/")
annotations_dir_path = Path("/pml/annotations/")

# The analysis path being an output remains to be local
analysis_dir_path = '~/dev/pml-echino/analysis/'

tiles_path = "/pml/"

# Storing pathml slides (result of tissue detection) outside, as they don`t change if images and annotations are the same
pathml_slide_dir_path = Path("/pml/slides")

os.makedirs(analysis_dir_path, exist_ok=True)
os.makedirs(os.path.join(analysis_dir_path, 'classification_results'), exist_ok=True)
os.makedirs(os.path.join(analysis_dir_path, 'segmentation_results'), exist_ok=True)

# Splitting train, validation and test dataset randomly
train_cases, val_cases, test_cases = caseRandomizer()

train_wsi_paths = [os.path.join(wsi_path, train_case+'.tif') for train_case in train_cases]
val_wsi_paths = [os.path.join(wsi_path, val_case+'.tif') for val_case in val_cases]
test_wsi_paths = [os.path.join(wsi_path, test_case+'.tif') for test_case in test_cases]


print("Paths are all set, dataset splitting done", "\r\n")

Loaded 92 images from /pml/wsi_data 

Paths are all set, dataset splitting done 



In [ ]:
def tissueDetection(tile_size):
    start_time = time.time()

    for wsi_path in train_wsi_paths+val_wsi_paths+test_wsi_paths:
        case = Path(wsi_path).stem
        # check for existing pml files
        case_pml_file = pathml_slide_dir_path / Path(case+'.pml')
        # print(f"DEBUG: {type(case_pml_file)}, {case_pml_file}")
        if case_pml_file.is_file():
            print(f"pml file exists for {case} as {case_pml_file}")
            continue
        
        pathml_slide = Slide(wsi_path, level=0).setTileProperties(tileSize=tile_size)
        pathml_slide.detectTissue(numWorkers=0) 
        pathml_slide.detectForeground()

        if 'echino' in case:
            annotation_path = os.path.join(annotations_dir_path, case+'.geojson')
            
            print(annotation_path)
            pathml_slide.addAnnotations(annotation_path, negativeClass='negative')

        pathml_slide.save(folder=pathml_slide_dir_path)

    time_elapsed = time.time() - start_time
    print('Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [5]:
def detectionchecking():
    for case in val_cases:
        pathml_slide = Slide(os.path.join(pathml_slide_dir_path, case+'.pml'))
        pathml_slide.visualizeThumbnail(folder=os.path.join(analysis_dir_path, 'classification_results'))
        pathml_slide.visualizeTissueDetection(folder=os.path.join(analysis_dir_path, 'classification_results'))

In [6]:
def tilextractor(tile_size):
    
    print("Tile size is set to", tile_size, "pixels")
    
    numTilesToExtract = 500

    start_time = time.time()
    
    global global_channel_sums
    global global_channel_squared_sums
    global global_tile_count

    global_channel_sums = np.zeros(3)
    global_channel_squared_sums = np.zeros(3)
    global_tile_count = 0

    for case in train_cases + val_cases:
        
        print(case)
               
        slide_file = str(pathml_slide_dir_path / Path(case+'.pml'))
        # print("slide file: ", slide_file)
        pathml_slide = Slide(slide_file)

        if 'echino' in case: #####change to echino
            channel_data = pathml_slide.extractAnnotationTiles(tiles_path, 
                                                               tileAnnotationOverlapThreshold=0.7,
                                                               otherClassNames='non_echinococcus',
                                                               numTilesToExtractPerClass=numTilesToExtract, 
                                                               extractSegmentationMasks=True, 
                                                               tissueLevelThreshold=0.995, 
                                                               foregroundLevelThreshold=88)

            global_channel_sums = np.add(global_channel_sums, channel_data['channel_sums'])
            global_channel_squared_sums = np.add(global_channel_squared_sums, channel_data['channel_squared_sums'])
            global_tile_count = global_tile_count + channel_data['num_tiles']

        else: 
            channel_data = pathml_slide.extractRandomUnannotatedTiles(tiles_path, 
                                                                      numTilesToExtract=numTilesToExtract, 
                                                                      otherClassNames='Tumor',
                                                                      unannotatedClassName='non_echinococcus',
                                                                      extractSegmentationMasks=True, 
                                                                      tissueLevelThreshold=0.995, 
                                                                      foregroundLevelThreshold=88)

            global_channel_sums = np.add(global_channel_sums, channel_data['channel_sums'])
            global_channel_squared_sums = np.add(global_channel_squared_sums, channel_data['channel_squared_sums'])
            global_tile_count = global_tile_count + channel_data['num_tiles']
            
    %store global_channel_sums
    %store global_channel_squared_sums
    %store global_tile_count

    time_elapsed = time.time() - start_time
    print('Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [7]:
def tileworks(batch_size):
    
    total_pixels_per_channel = global_tile_count * tile_size * tile_size
    global_channel_means = np.divide(global_channel_sums, total_pixels_per_channel)
    global_channel_squared_means = np.divide(global_channel_squared_sums, total_pixels_per_channel)
    global_channel_variances = np.subtract(global_channel_squared_means, np.square(global_channel_means))
    global_channel_stds = np.sqrt(global_channel_variances * (total_pixels_per_channel / (total_pixels_per_channel-1)))
    
    global means_and_stds
    
    means_and_stds = {'channel_means': global_channel_means.tolist(), 'channel_stds': global_channel_stds.tolist()}
    
    pickle.dump(means_and_stds,
        open(os.path.join(analysis_dir_path, 'classification_results', 'trainval_channel_means_and_stds.p'), 'wb'))

    print('Channel means:', global_channel_means.tolist())
    print('Channel standard deviations:', global_channel_stds.tolist())

    echninococcus_tiles = glob.glob(os.path.join(tiles_path, 'tiles', '*', 'Tumor', '*.jpg'))
    non_echninococcus_tiles = glob.glob(os.path.join(tiles_path, 'tiles', '*', 'non_echinococcus', '*.jpg'))

    print('Total number of echinococcus tiles:', len(echninococcus_tiles))
    print('Total number of non-echninococcus tiles:', len(non_echninococcus_tiles))

    data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0, contrast=0, saturation=1, hue=.5),
        transforms.ToTensor(),
        transforms.Normalize(means_and_stds['channel_means'], means_and_stds['channel_stds'])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(means_and_stds['channel_means'], means_and_stds['channel_stds'])
    ]),
    }
    
    global dataset_sizes
    global dataloaders
    
    train_dataset = torch.utils.data.ConcatDataset([datasets.ImageFolder(os.path.join(
            tiles_path, 'tiles', train_case), data_transforms['train']) for train_case in train_cases])
    val_dataset = torch.utils.data.ConcatDataset([datasets.ImageFolder(os.path.join(
            tiles_path, 'tiles', val_case), data_transforms['val']) for val_case in val_cases])

    image_datasets = {'train': train_dataset, 'val': val_dataset}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    
    dataloaders = {}
    batch_size = 48
    dataloaders['train'] = torch.utils.data.DataLoader(
        image_datasets['train'], batch_size=batch_size, num_workers=0, shuffle=True)
    dataloaders['val'] = torch.utils.data.DataLoader(
        image_datasets['val'], batch_size=batch_size, num_workers=0, shuffle=True)

    %store dataloaders
    %store means_and_stds
    %store dataset_sizes
  

In [8]:
def vizBatch():

    def visualize_batch(inp, title=None):
        inp = inp.numpy().transpose((1, 2, 0))
        mean = means_and_stds['channel_means']
        std = means_and_stds['channel_stds']
        inp = std * inp + mean
        inp = np.clip(inp, 0, 1)
        plt.imshow(inp)
        if title is not None:
            plt.title(title)
        plt.pause(0.001)

    inputs, classes = next(iter(dataloaders['train']))
    out = torchvision.utils.make_grid(inputs)
    visualize_batch(out)

In [9]:
def train_classification_model(model, criterion, optimizer, scheduler=False, num_epochs=30):
    best_model_state_dict = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0
    learning_stats = {'train': [], 'val': []}
        
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            epoch_ground_truth = []
            epoch_predictions = []
            running_loss = 0.0#
            
           
        
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # print(labels)
                
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                      
                epoch_ground_truth = epoch_ground_truth + labels.data.tolist()
                epoch_predictions = epoch_predictions + preds.tolist()
                running_loss += loss.item() * inputs.size(0)
                
            if scheduler:
                if phase == 'train':
                    scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = accuracy_score(epoch_ground_truth, epoch_predictions)
            epoch_weighted_acc = balanced_accuracy_score(epoch_ground_truth, epoch_predictions)
            epoch_weighted_rec = recall_score(epoch_ground_truth, epoch_predictions, average='weighted')
            epoch_weighted_prec = precision_score(epoch_ground_truth, epoch_predictions, average='weighted')
            epoch_weighted_f1 = f1_score(epoch_ground_truth, epoch_predictions, average='weighted')
            
            learning_stats[phase].append(
                {'loss': epoch_loss, 
                 'accuracy': epoch_acc, 
                 'weighted_accuracy': epoch_weighted_acc,
                 'weighted_precision': epoch_weighted_prec,
                 'weighted_recall': epoch_weighted_rec,
                 'weighted_f1': epoch_weighted_f1})
            
            print('Phase Loss: {:.4f} Acc: {:.4f} Weighted Acc: {:.4f} Weighted Pre: {:.4f} Weighted Rec: {:.4f} Weighted F1: {:.4f}')
            print(phase, epoch_loss, epoch_acc, epoch_weighted_acc, epoch_weighted_prec, epoch_weighted_rec, epoch_weighted_f1)
     
            
            if (phase == 'val') and (epoch_acc > best_accuracy):
                best_accuracy = epoch_acc
                best_model_state_dict = copy.deepcopy(model.state_dict())
                  
    return best_model_state_dict, learning_stats

**Plot classification learning curve**

In [10]:
def plotLearningCurve():
    trainLoss = [epoch["loss"] for epoch in learningStats['train']]
    valLoss = [epoch["loss"] for epoch in learningStats['val']]
    trainAcc = [epoch["weighted_accuracy"] for epoch in learningStats['train']]
    valAcc = [epoch["weighted_accuracy"] for epoch in learningStats['val']]
    numEpochs = len(learningStats['train'])

    fig, (ax1, ax2) = plt.subplots(2)
    fig.suptitle('Echinococcus classification')
    ax1.plot(np.arange(numEpochs)+1,trainAcc,'bo-.',label="Training",alpha=0.6,markersize=4)
    ax1.plot(np.arange(numEpochs)+1,valAcc,'go-',label="Validation",markersize=4)
    ax1.axhline(y=np.max(valAcc),color="r",alpha=0.4)
    ax1.set(ylabel="Weighted accuracy")
    ax1.label_outer()
    ax2.plot(np.arange(numEpochs)+1,trainLoss,'bo-.',label="Training",alpha=0.6,markersize=4)
    ax2.plot(np.arange(numEpochs)+1,valLoss,'go-',label="Validation",markersize=4)
    ax2.set(xlabel="Epoch", ylabel="Loss")
    fig.set_size_inches(7,9)
    plt.legend()
    plt.savefig(os.path.join(analysis_dir_path, 'classification_results', 'classification_learning_curves.png'))
    plt.show(block=False)

In [11]:
def vizInferHeatmap_Val():
    print("\r\n Visualizing val cases\r\n")
    for case in val_cases:
        pathml_slide = Slide(os.path.join(pathml_slide_dir_path, case+'.pml'))
        pathml_slide.visualizeClassifierInference('Tumor', folder=os.path.join(analysis_dir_path, 'classification_results'))

def vizInferHeatmap_Test():
    print("\r\n Visualizing test cases\r\n")
    
    for case in test_cases:
        pathml_slide = Slide(os.path.join(pathml_slide_dir_path, case+'.pml'))
        pathml_slide.visualizeClassifierInference('Tumor', folder=os.path.join(analysis_dir_path, 'classification_results'))

In [12]:
def probThresholds():   
    
    global best_classification_threshold
    
    probability_thresholds = np.append(np.arange(0, 1, 0.005),[0.9925, 0.995, 0.9975, 0.999, 0.99925, 0.9995, 0.99975, 0.9999, 0.999925, 0.99995, 0.999975, 0.99999, 0.9999925, 0.999995, 0.9999975, 0.999999]).tolist()
    threshold_accuracies_all_slides = []

    for val_case in val_cases:
        val_slide_path = os.path.join(pathml_slide_dir_path, val_case+'.pml')
        val_slide = Slide(val_slide_path)

        threshold_accuracies = val_slide.classifierMetricAtThreshold('Tumor', probability_thresholds, tileAnnotationOverlapThreshold=0.3, metric='accuracy', assignZeroToTilesWithoutAnnotationOverlap=True)
        threshold_accuracies_all_slides.append(threshold_accuracies)

    threshold_avg_accuracies = np.mean(np.array(threshold_accuracies_all_slides), axis=0)
    index_of_best_classification_threshold = np.argmax(threshold_avg_accuracies)
    best_classification_threshold = probability_thresholds[index_of_best_classification_threshold]

    print("Best tile-level validation accuracy:", threshold_avg_accuracies[index_of_best_classification_threshold])
    print("Threshold that gives the best tile-level validation accuracy:", best_classification_threshold)
    
    plt.figure(figsize=(7.5,6))
    plt.plot(probability_thresholds, threshold_avg_accuracies)
    plt.xlim(-0.05, 1.05)
    plt.ylim(0.6, 1.0)
    plt.title('Tile-level accuracy on val vs. tile probability threshold')
    plt.xlabel('Probability threshold for determination\nof number of tiles showing echinococcus')
    plt.ylabel('Tile accuracy for echinococcus detection with\nthresholded number of tiles')
    plt.savefig(os.path.join(analysis_dir_path, 'classification_results', 'classification_validation_tile_accuracy_vs_probability_threshold.png'))
    plt.show(block=False)
    
    %store best_classification_threshold

**Test set accuracy by threshold**

In [13]:
def testaccuracy():

    test_accuracies_df_rows = []

    for test_case in test_cases:
        test_slide_path = os.path.join(pathml_slide_dir_path, test_case+'.pml')
        test_slide = Slide(test_slide_path)
        test_accuracy = test_slide.classifierMetricAtThreshold('Tumor', best_classification_threshold, tileAnnotationOverlapThreshold=0.3, metric='accuracy', assignZeroToTilesWithoutAnnotationOverlap=True)
        test_accuracies_df_rows.append([test_case, test_accuracy])

    test_accuracies_df = pd.DataFrame(test_accuracies_df_rows, columns=['Slide_ID', 'echinococcus accuracy (>'+str(round(best_classification_threshold, 6))+')'])
    test_accuracies_df.to_csv(os.path.join(analysis_dir_path, 'classification_results', 'classification_test_accuracies_with_'+str(round(best_classification_threshold, 6))+'_df.csv'), index=False)

    print("Average test set accuracy at echinococcus probability threshold of "+str(best_classification_threshold)+":", test_accuracies_df['echinococcus accuracy (>'+str(round(best_classification_threshold, 6))+')'].mean())
    print("Case-by-case test set accuracies:")
    print(test_accuracies_df)


**Slide level tile counts above threshold**

In [14]:
def tilecountslidelevel():
    
    global test_tile_counts_df
    test_tile_counts_df_rows = []

    for test_case in test_cases:
        test_slide = Slide(os.path.join(pathml_slide_dir_path, test_case+'.pml'))
        num_metastasis_tiles_above_threshold = test_slide.numTilesAboveClassPredictionThreshold('Tumor', best_classification_threshold)

        if 'echino' in test_case:
            metastasis_ground_truth = 1
        else:
            metastasis_ground_truth = 0

        test_tile_counts_df_rows.append([test_case, metastasis_ground_truth, num_metastasis_tiles_above_threshold])

    test_tile_counts_df = pd.DataFrame(test_tile_counts_df_rows, columns=['Slide_ID', 'echinococcus_ground_truth', 'echinococcus tile count (>'+str(round(best_classification_threshold, 6))+')'])
    test_tile_counts_df.to_csv(os.path.join(analysis_dir_path, 'classification_results', 'classification_test_tile_counts_above_'+str(round(best_classification_threshold, 6))+'_df.csv'), index=False)

    print(test_tile_counts_df)
    
    %store test_tile_counts_df

**AUC**

In [15]:
def finalAUC():
    testAuc = roc_auc_score(test_tile_counts_df['echinococcus_ground_truth'], 
        test_tile_counts_df['echinococcus tile count (>'+str(round(best_classification_threshold, 6))+')'])
    fpr, tpr, thresholds = roc_curve(test_tile_counts_df['echinococcus_ground_truth'], 
        test_tile_counts_df['echinococcus tile count (>'+str(round(best_classification_threshold, 6))+')'])

    plt.figure()
    plt.title('ROC with echinococcus probability threshold of '+str(round(best_classification_threshold, 6)))
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % testAuc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig(os.path.join(analysis_dir_path, 'classification_results', 'classification_test_auc_roc.png'))
    plt.show(block=False)